In [ ]:
# Clone GitHub repository
!git clone https://github.com/zlisto/social_media_marketing

import os
os.chdir("social_media_marketing")


In [ ]:
# Install requirements
!pip install openai


In [1]:
import pandas as pd
import datetime
import openai
import textwrap as tr
import os 
import time
import ast

pd.set_option("display.max_colwidth", None)

In [2]:
#set your OpenAI API key as a string
OPENAI_API_KEY= ''



In [3]:
client = openai.Client(api_key=OPENAI_API_KEY)


# Create YaleSOM Tweets

In [4]:
instruction_file_path = "data/instructions_YaleSOM.txt"
with open(instruction_file_path, 'r', encoding='utf-8') as file:
    instructions = file.read()


In [5]:
print(instructions)

You are going to act like the YaleSOM Twitter account for the Yale School of Management.  Some top tweets from the account are provided at the end.  Learn from theset tweets what types of tweets are good and the style of YaleSOM tweeting.  You will be asked by the user to write engaging tweets in the voice of this account. You may also be asked to write the tweet on a user specificed topic.  Provide the tweet in your response and make the tweet be engaging, based on your analysis of past YaleSOM tweets provided here.  Your response should be the tweet, a new line, and then a short explanation why it will be engaging. If the topic is not represented in these tweets, use your best knowledge to craft the tweet. Here are some YaleSOM tweets for you to study:



"Prof. @JeffSonnenfeld on whether Elon Musk's eccentricities are undermining his leadership of Twitter, Tesla, and SpaceX.

Read more in #Yaleinsights: https://t.co/LYGPdPJJaG"
"Want to learn about antitrust, but don't know where to

In [6]:
#create assistant
assistant_name = 'YaleSOM GPT'
#model = 'gpt-3.5-turbo-1106'
model="gpt-4-1106-preview"
print(f"{assistant_name} Model {model}")

assistant = client.beta.assistants.create(
    name=assistant_name,
    instructions=instructions,
    model=model
)



YaleSOM GPT Model gpt-4-1106-preview


In [7]:
#function to chat with assistant

def get_chatbot_response(user_input, thread):
    # This is where you'd integrate your chatbot logic
    print("Getting GPT response...")
    message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input,
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

        if run_status.completed_at is not None:
            break

        time.sleep(1)  # Sleep for a second

    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )

    # Assistant's response
    msg = messages.data[0]
    
    if msg.role == 'assistant':
        for content in msg.content:
            if content.type == 'text':
                response  = content.text.value
            else:
                response = "No text in response"

    return f"{response}" 

In [8]:
thread = client.beta.threads.create()  #initialize the conversation thread


In [9]:
#chat with assistant
while True:
    user_input = input("\nYou: ")
    if user_input.lower() == 'quit':
        break
    response = get_chatbot_response(user_input, thread)
    print(tr.fill(f"\n{assistant_name}: {response}", width = 75))


You: write a tweet about Yale being better than Harvard and be funny.
Getting GPT response...
 YaleSOM GPT: "Just spotted a bulldog outsmarting a crimson tie on campus!
Looks like even our four-legged friends know @YaleSOM > Harvard. 😉
#IvyLeagueRivalry #SmartDogsChooseYale https://t.co/YalePawsAhead"  The
tweet uses humor by personifying a bulldog, Yale's mascot, as smarter than
Harvard, represented by its color crimson. It playfully indulges in the Ivy
League rivalry while staying lighthearted. It also includes a mock hashtag
to engage the audience and a fictional humorous URL encouraging a sense of
Yale pride.

You: quit


# Image Generation

In [10]:
#Give instructions to the AI
instruction = "you will help create cool looking sneakers for a high end fashion company catering to hip young people."
instruction+="You return the answers as prompts for Dalle-3 as JSON output with keys prompt1, prompt2, etc."

#Give your task for the AI
user_input =  "Give me 5 ideas for images of sneaers that we can post on the companys instagram."



In [12]:
#get the response from the AI
#model = 'gpt-3.5-turbo-1106'
model="gpt-4-1106-preview"

responses = client.chat.completions.create(
  model=model,
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": instruction},
    {"role": "user", "content": user_input}
  ]
)

response = responses.choices[0].message.content
print(response)


{
    "prompt1": "Create an image of a sneaker with holographic leather and neon green accents, showcasing advanced tech vibes with a futuristic background.",
    "prompt2": "Design a sneaker with elements of street graffiti art, vibrant colors on a jet black base, against an urban brick wall background with a mural.",
    "prompt3": "Imagine a sneaker incorporating sustainable materials, with natural green and earthy tones, positioned in a lush forest environment for an eco-friendly aesthetic.",
    "prompt4": "Visualize a high-end sneaker with a glossy patent leather finish, gold hardware, and intricate detailing, placed on a pedestal under a spotlight to convey luxury.",
    "prompt5": "Generate an image of a sneaker with an interactive LED display integrated into the design, custom animations playing across the shoe, in a dimly-lit setting to highlight the feature."
}


In [13]:
#convert the text response into a json object
prompt_json = ast.literal_eval(response.strip())


In [14]:
#generate the image for each prompt
for cnt,prompt in enumerate(prompt_json.values()):
    print(tr.fill(f"{cnt+1}. {prompt}", width = 75))
    response_img = client.images.generate(
      model="dall-e-3",
      prompt=prompt,
      size="1024x1024",
      quality="standard",
      n=1,
    )
    time.sleep(1)

    image_url = response_img.data[0].url
    print(image_url)

1. Create an image of a sneaker with holographic leather and neon green
accents, showcasing advanced tech vibes with a futuristic background.
https://oaidalleapiprodscus.blob.core.windows.net/private/org-zrRGnR9FWbvUcS3XEz3mQZCr/user-l84oQyDEGesRPUmssaSpPuwX/img-fQMv78Uq1dKfznr31afp8DWT.png?st=2023-11-29T07%3A22%3A46Z&se=2023-11-29T09%3A22%3A46Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-11-28T20%3A03%3A04Z&ske=2023-11-29T20%3A03%3A04Z&sks=b&skv=2021-08-06&sig=4bp%2BwpONPPr2osTc/aVfBCym9yHSQfgxyK0S7HuOn00%3D
2. Design a sneaker with elements of street graffiti art, vibrant colors on
a jet black base, against an urban brick wall background with a mural.
https://oaidalleapiprodscus.blob.core.windows.net/private/org-zrRGnR9FWbvUcS3XEz3mQZCr/user-l84oQyDEGesRPUmssaSpPuwX/img-i6M4zUgdH0Zw794gipGMo9Cc.png?st=2023-11-29T07%3A22%3A58Z&se=2023-11-29T09%3A22%3A58Z&sp=r&sv=2021-08-06&sr=b&rscd=i